In [61]:
import time
import hashlib
import pandas as pd

In [77]:
'''
Setting up the input data for the blocks 
Each row in this dataframe corresponds to a new block in the chain
The dataframe will also store the metadata about block once we are done mining each bloc
'''
input_data = pd.DataFrame(columns={'data','data_id','prev_block_id','curr_block_id', 'mining_time','nonce'})
input_data['data'] = ["Let's create the first block in the chain", "This is the second block", "This is the third block", "This is the fourth block"]
input_data

,prev_block_id,nonce,curr_block_id,data,mining_time,data_id
0,NaN,NaN,NaN,Let's create the first block in the chain,NaN,NaN
1,NaN,NaN,NaN,This is the second block,NaN,NaN
2,NaN,NaN,NaN,This is the third block,NaN,NaN
3,NaN,NaN,NaN,This is the fourth block,NaN,NaN


In [141]:
#How many leading zeros count (lzc) should be there in the acceptable hash for current block ID
'''User can change this, please note that the larger this number the longer it will take for the program to finish running'''
lzc = 6 #Keep it striictly greater than 0

In [142]:
'''
This function is called for each row (therefore each block)
::input::
data_to_hash: the data to be stored in the block
prev_block_id: the previous block's id, for the first block it will be empty
::output::
data_id: the Data ID for the datain the block
prev_block_id: the previous block ID
current_block_id: the current block ID
nonce: the nonce value, which also represent the number of attempts it took to mine the block
mining_time: the time in seconds that it took to mine the block
'''
def create_block(data_to_hash, prev_block_id=None):
    result = hashlib.md5(data_to_hash.encode())
    data_id = result.hexdigest()
    nonce = 0
    start_time = time.time()
    hash_output = data_id
    if hash_output[:lzc]!='0'*lzc:
        while hash_output[:lzc]!='0'*lzc:
            if prev_block_id:
                nonced_data = prev_block_id+'\n'+data_id+'\n'+str(nonce)
            else:
                nonced_data = data_id+'\n'+str(nonce)
            result = hashlib.md5(nonced_data.encode())
            hash_output = result.hexdigest()
            nonce = nonce+1
    current_block_id = hash_output
    mining_time = time.time() - start_time
    nonce
    return data_id, prev_block_id, current_block_id, mining_time, nonce

In [143]:
'''Calling the block mining function for each row of the input'''
for index, row in input_data.iterrows():
    data_to_hash = row['data']
    if index!=0:
        prev_block_id = input_data.at[index-1,'curr_block_id']
        mining_output = create_block(data_to_hash,prev_block_id)
    else:
        mining_output = create_block(data_to_hash)
    row['data_id'] = mining_output[0]
    row['prev_block_id'] = mining_output[1]
    row['curr_block_id'] = mining_output[2]
    row['mining_time'] = mining_output[3]
    row['nonce'] = mining_output[4]

In [144]:
'''Display the output once all blocks are mined'''
input_data[['mining_time','nonce','data','data_id','prev_block_id','curr_block_id']]

,mining_time,nonce,data,data_id,prev_block_id,curr_block_id
0,7.28845,4811494,Let's create the first block in the chain,bcfa87c919c4e7710975408c8971a5b1,None,0000008f6d50d28ce23971c27b63eb65
1,14.921,8756689,This is the second block,f983c34da708267a231b4b550d89090a,0000008f6d50d28ce23971c27b63eb65,00000091b3988c70492fe20e0f2d3dfe
2,72.8699,42714425,This is the third block,5017c5fbf82a92b6ad94fb827c90390f,00000091b3988c70492fe20e0f2d3dfe,0000004af80cb1c93966b7084b75930e
3,8.39357,4888438,This is the fourth block,f2bfa3483b3d134247054ba9e7d2d7dc,0000004af80cb1c93966b7084b75930e,0000004ef978a29ae3188eecd781a62e
